In [ ]:
output_path = '/content/merged_cases_random_10k.csv'

with open(output_path, 'w', encoding='utf-8') as outfile:
    for i, path in enumerate([
        '/content/cases_random_0kto1k.csv',
        '/content/cases_random_1kto2k.csv',
        '/content/cases_random_2kto3k.csv',
        '/content/cases_random_3kto4k.csv',
        '/content/cases_random_4kto5k.csv',
        '/content/cases_random_5kto6k.csv',
        '/content/cases_random_6kto7k.csv',
        '/content/cases_random_7kto8k.csv',
        '/content/cases_random_8kto9k.csv',
        '/content/cases_random_9kto10k.csv',
    ]):
        with open(path, 'r', encoding='utf-8') as infile:
            lines = infile.readlines()
            if i == 0:
                outfile.writelines(lines)
            else:
                outfile.writelines(lines[1:])



✅ Готово: /content/merged_cases_random_10k.csv


In [ ]:
import pandas as pd

main_df = pd.read_csv('/content/merged_cases_random_10k.csv')


predicted_files = [
    '/content/predicted_result_type_0k1k.csv',
    '/content/predicted_result_type_1k2k.csv',
    '/content/predicted_result_type_2k3k.csv',
    '/content/predicted_result_type_3k4k.csv',
    '/content/predicted_result_type_4k5k.csv',
    '/content/predicted_result_type_5k6k.csv',
    '/content/predicted_result_type_6k7k.csv',
    '/content/predicted_result_type_7k8k.csv',
    '/content/predicted_result_type_8k9k.csv',
    '/content/predicted_result_type_9k10k.csv',
]

predicted_dfs = [pd.read_csv(path) for path in predicted_files]
predicted_df = pd.concat(predicted_dfs, ignore_index=True)

assert 'id' in main_df.columns, 'Нет столбца id в основном датафрейме'
assert 'id' in predicted_df.columns and 'predicted_result_type' in predicted_df.columns, 'Нет нужных колонок в predicted'

main_df['id'] = main_df['id'].astype(str)
predicted_df['id'] = predicted_df['id'].astype(str)

merged = pd.merge(main_df, predicted_df[['id', 'predicted_result_type']], on='id', how='left')

cols = list(merged.columns)
cols.insert(1, cols.pop(cols.index('predicted_result_type')))
merged = merged[cols]


merged.to_csv('/content/merged_with_prediction.csv', index=False)


✅ Готово. Файл сохранён: /content/merged_with_prediction.csv


In [ ]:
import pandas as pd

merged = pd.read_csv('/content/merged_with_prediction.csv')

parsed_files = [
    '/content/parsed_sentences.csv0kto1k.csv',
    '/content/parsed_sentences.csv1kto2.csv',
    '/content/parsed_sentences.csv2kto3k.csv',
    '/content/parsed_sentences.csv3kto4k.csv',
    '/content/parsed_sentences.csv4kto5k.csv',
    '/content/parsed_sentences.csv5kto6k.csv',
    '/content/parsed_sentences.csv6kto7k.csv',
    '/content/parsed_sentences.csv7kto8k.csv',
    '/content/parsed_sentences.csv8kto9k.csv',
    '/content/parsed_sentences.csv9kto10k.csv',
]

parsed_dfs = [pd.read_csv(path) for path in parsed_files]
parsed_df = pd.concat(parsed_dfs, ignore_index=True)

merged['id'] = merged['id'].astype(str)
parsed_df['id'] = parsed_df['id'].astype(str)

parsed_df = parsed_df.rename(columns={
    'Sentence year': 'sentence_year',
    'Sentence month': 'sentence_months',
    'Sentence day': 'sentence_days',
    'Sentence hour': 'sentence_hours',
    'Is suspended': 'is_suspended',
    'Suspention years': 'suspension_years',
    'Suspention months': 'suspension_months',
    'Is_fixed_fine': 'fine_is_fixed',
    'Fine amount': 'fine_amount'
})

columns_to_add = [
    'id',
    'sentence_year',
    'sentence_months',
    'sentence_days',
    'sentence_hours',
    'is_suspended',
    'suspension_years',
    'suspension_months',
    'fine_is_fixed',
    'fine_amount'
]

parsed_df = parsed_df[columns_to_add]

merged = pd.merge(merged, parsed_df, on='id', how='left')

before_cols = list(merged.columns)
target_index = before_cols.index('predicted_result_type') + 1

new_order = (
    before_cols[:target_index] +
    [col for col in columns_to_add if col != 'id'] +
    [col for col in before_cols if col not in ['predicted_result_type'] + [col for col in columns_to_add if col != 'id']]
)

merged = merged[new_order]

merged.to_csv('/content/merged_with_prediction_and_sentences.csv', index=False)

✅ Готово! Файл: /content/merged_with_prediction_and_sentences.csv


In [ ]:

merged = pd.read_csv('/content/merged_with_prediction_and_sentences.csv')

drug_files = [
    '/content/predicted_0kto1k_grams.csv',
    '/content/predicted_1kto2k_grams.csv',
    '/content/predicted_2kto3k_grams.csv',
    '/content/predicted_3kto4k_grams.csv',
    '/content/predicted_4kto5k_grams.csv',
    '/content/predicted_5kto6k_grams.csv',
    '/content/predicted_6kto7k_grams.csv',
    '/content/predicted_7kto8k_grams.csv',
    '/content/predicted_8kto9k_grams.csv',
    '/content/predicted_9kt10k_grams.csv',
]

drug_dfs = [pd.read_csv(path) for path in drug_files]
drug_df = pd.concat(drug_dfs, ignore_index=True)

merged['id'] = merged['id'].astype(str)
drug_df['id'] = drug_df['id'].astype(str)

merged = pd.merge(merged, drug_df, on='id', how='left')

before_cols = list(merged.columns)
target_index = before_cols.index('fine_amount') + 1

new_order = (
    before_cols[:target_index] +
    ['drug_weight'] +
    [col for col in before_cols if col not in ['drug_weight']]
)

merged = merged[new_order]
merged.to_csv('/content/final_merged_all.csv', index=False)


✅ Готово! Финальный файл: /content/final_merged_all.csv


In [ ]:
df = pd.read_csv('/content/final_merged_all.csv')

df = df.loc[:, ~df.columns.duplicated()]
df = df[[col for col in df.columns if not any(suffix in col for suffix in ['.1', '.2'])]]

df.to_csv('/content/final_clean.csv', index=False)

print(df.columns.tolist())

✅ Готово! Чистый датасет без дубликатов сохранён в: /content/final_clean.csv
['id', 'predicted_result_type', 'sentence_year', 'sentence_months', 'sentence_days', 'sentence_hours', 'is_suspended', 'suspension_years', 'suspension_months', 'fine_is_fixed', 'fine_amount', 'drug_weight', 'entry_date', 'result_date', 'in_favorites', 'court', 'codex_articles', 'defendants_simple', 'penalty', 'penalty_type', 'penalty_value', 'result_type', 'result_text', 'result_text_url', 'court_city', 'region', 'type', 'stage', 'url', 'appeal_date', 'defendants_gender', 'judge', 'case_number', 'last_scheduled_hearing_date', 'sm_url']


In [ ]:
print("Уникальные регионы:")
print(sorted(df['region'].dropna().unique()))

Уникальные регионы:
['Алтайский край', 'Архангельская область', 'Астраханская область', 'Белгородская область', 'Брянская область', 'Владимирская область', 'Волгоградская область', 'Вологодская область', 'Воронежская область', 'Город Москва', 'Город Санкт-Петербург', 'Город Севастополь', 'Еврейская автономная область', 'Забайкальский край', 'Ивановская область', 'Иркутская область', 'Кабардино-Балкарская Республика', 'Калининградская область', 'Калужская область', 'Камчатский край', 'Кемеровская область', 'Кировская область', 'Костромская область', 'Краснодарский край', 'Красноярский край', 'Курганская область', 'Курская область', 'Ленинградская область', 'Липецкая область', 'Магаданская область', 'Московская область', 'Мурманская область', 'Ненецкий автономный округ', 'Нижегородская область', 'Новгородская область', 'Новосибирская область', 'Омская область', 'Оренбургская область', 'Орловская область', 'Пензенская область', 'Пермский край', 'Приморский край', 'Псковская область', 'Рес

In [ ]:
region_coords = {
    'Алтайский край': {'latitude': 53.3606, 'longitude': 83.7636},
    'Архангельская область': {'latitude': 64.5401, 'longitude': 40.5433},
    'Астраханская область': {'latitude': 46.3497, 'longitude': 48.0408},
    'Белгородская область': {'latitude': 50.767, 'longitude': 37.45},
    'Брянская область': {'latitude': 52.95, 'longitude': 33.4},
    'Владимирская область': {'latitude': 56.083, 'longitude': 40.617},
    'Волгоградская область': {'latitude': 49.733, 'longitude': 44.117},
    'Вологодская область': {'latitude': 60.083, 'longitude': 40.45},
    'Воронежская область': {'latitude': 51.05, 'longitude': 40.15},
    'Город Москва': {'latitude': 55.7558, 'longitude': 37.6178},
    'Город Санкт-Петербург': {'latitude': 59.9343, 'longitude': 30.3351},
    'Город Севастополь': {'latitude': 44.6167, 'longitude': 33.5254},
    'Еврейская автономная область': {'latitude': 48.7946, 'longitude': 132.9218},
    'Забайкальский край': {'latitude': 52.0333, 'longitude': 113.5},
    'Ивановская область': {'latitude': 56.9972, 'longitude': 40.9714},
    'Иркутская область': {'latitude': 52.2978, 'longitude': 104.2964},
    'Кабардино-Балкарская Республика': {'latitude': 43.4858, 'longitude': 43.607},
    'Калининградская область': {'latitude': 54.7104, 'longitude': 20.4522},
    'Калужская область': {'latitude': 54.5293, 'longitude': 36.2754},
    'Камчатский край': {'latitude': 53.0451, 'longitude': 158.6483},
    'Кемеровская область': {'latitude': 55.354, 'longitude': 86.0879},
    'Кировская область': {'latitude': 58.6035, 'longitude': 49.6678},
    'Костромская область': {'latitude': 57.7679, 'longitude': 40.9269},
    'Краснодарский край': {'latitude': 45.0355, 'longitude': 38.9753},
    'Красноярский край': {'latitude': 56.0153, 'longitude': 92.8932},
    'Курганская область': {'latitude': 55.4443, 'longitude': 65.3161},
    'Курская область': {'latitude': 51.7304, 'longitude': 36.1926},
    'Ленинградская область': {'latitude': 59.9386, 'longitude': 30.3141},
    'Липецкая область': {'latitude': 52.6031, 'longitude': 39.5708},
    'Магаданская область': {'latitude': 59.5682, 'longitude': 150.8085},
    'Московская область': {'latitude': 55.7558, 'longitude': 37.6178},
    'Мурманская область': {'latitude': 68.9792, 'longitude': 33.0925},
    'Ненецкий автономный округ': {'latitude': 67.6382, 'longitude': 53.0031},
    'Нижегородская область': {'latitude': 56.2965, 'longitude': 43.9361},
    'Новгородская область': {'latitude': 58.5215, 'longitude': 31.2755},
    'Новосибирская область': {'latitude': 55.0084, 'longitude': 82.9357},
    'Омская область': {'latitude': 54.9914, 'longitude': 73.3715},
    'Оренбургская область': {'latitude': 51.7682, 'longitude': 55.0968},
    'Орловская область': {'latitude': 52.9673, 'longitude': 36.0696},
    'Пензенская область': {'latitude': 53.175, 'longitude': 45.0348},
    'Пермский край': {'latitude': 58.0105, 'longitude': 56.2502},
    'Приморский край': {'latitude': 43.1155, 'longitude': 131.8855},
    'Псковская область': {'latitude': 57.8193, 'longitude': 28.3318},
    'Республика Адыгея': {'latitude': 44.6098, 'longitude': 40.1006},
    'Республика Алтай': {'latitude': 51.958, 'longitude': 85.9603},
    'Республика Башкортостан': {'latitude': 54.7388, 'longitude': 55.9721},
    'Республика Бурятия': {'latitude': 51.8335, 'longitude': 107.5842},
    'Республика Дагестан': {'latitude': 42.9849, 'longitude': 47.5047},
    'Республика Ингушетия': {'latitude': 43.1675, 'longitude': 44.8169},
    'Республика Калмыкия': {'latitude': 46.3083, 'longitude': 44.2706},
    'Республика Карачаево-Черкесия': {'latitude': 43.7731, 'longitude': 41.9239},
    'Республика Карелия': {'latitude': 61.7891, 'longitude': 34.3596},
    'Республика Коми': {'latitude': 61.6689, 'longitude': 50.8356},
    'Республика Крым': {'latitude': 44.9521, 'longitude': 34.1024},
    'Республика Марий Эл': {'latitude': 56.6344, 'longitude': 47.8999},
    'Республика Мордовия': {'latitude': 54.1808, 'longitude': 45.1869},
    'Республика Саха (Якутия)': {'latitude': 62.0272, 'longitude': 129.7326},
'Республика Северная Осетия-Алания': {'latitude': 43.0241, 'longitude': 44.6819},
    'Республика Татарстан': {'latitude': 55.7903, 'longitude': 49.1347},
    'Республика Тыва': {'latitude': 51.7191, 'longitude': 94.437},
    'Республика Хакасия': {'latitude': 53.7203, 'longitude': 91.4424},
    'Ростовская область': {'latitude': 47.2357, 'longitude': 39.7015},
    'Рязанская область': {'latitude': 54.6197, 'longitude': 39.744},
    'Самарская область': {'latitude': 53.1959, 'longitude': 50.1008},
    'Саратовская область': {'latitude': 51.5304, 'longitude': 46.0342},
    'Сахалинская область': {'latitude': 46.9591, 'longitude': 142.7381},
    'Свердловская область': {'latitude': 56.8389, 'longitude': 60.6057},
    'Смоленская область': {'latitude': 54.7826, 'longitude': 32.0453},
    'Ставропольский край': {'latitude': 45.0448, 'longitude': 41.969},
    'Тамбовская область': {'latitude': 52.7212, 'longitude': 41.4523},
    'Томская область': {'latitude': 56.501, 'longitude': 84.9925},
    'Тульская область': {'latitude': 54.193, 'longitude': 37.6177},
    'Тюменская область': {'latitude': 57.153, 'longitude': 65.5272},
    'Удмуртская Республика': {'latitude': 56.8526, 'longitude': 53.2045},
    'Ульяновская область': {'latitude': 54.3081, 'longitude': 48.3748},
    'Хабаровский край': {'latitude': 48.4827, 'longitude': 135.0838},
    'Ханты-Мансийский автономный округ-Югра': {'latitude': 61.0042, 'longitude': 69.0019},
    'Челябинская область': {'latitude': 55.1644, 'longitude': 61.4368},
    'Чеченская Республика': {'latitude': 43.3178, 'longitude': 45.6986},
    'Чувашская Республика': {'latitude': 56.1439, 'longitude': 47.2489},
    'Чукотский автономный округ': {'latitude': 64.7337, 'longitude': 177.5089},
    'Ямало-Ненецкий автономный округ': {'latitude': 66.53, 'longitude': 66.56},
    'Ярославская область': {'latitude': 57.6261, 'longitude': 39.8845}
}

In [ ]:
import pandas as pd

df = pd.read_csv('/content/final_clean.csv')

def get_latitude(region):
    return region_coords.get(region, {}).get('latitude')

def get_longitude(region):
    return region_coords.get(region, {}).get('longitude')

df['latitude'] = df['region'].apply(get_latitude)
df['longitude'] = df['region'].apply(get_longitude)

df.to_csv('/content/final_with_coords.csv', index=False)


✅ Координаты добавлены и сохранены в 'final_with_coords.csv'


In [ ]:
import pandas as pd

df = pd.read_csv('/content/final_with_coords.csv')
print(df.columns.tolist())

['id', 'predicted_result_type', 'sentence_year', 'sentence_months', 'sentence_days', 'sentence_hours', 'is_suspended', 'suspension_years', 'suspension_months', 'fine_is_fixed', 'fine_amount', 'drug_weight', 'entry_date', 'result_date', 'in_favorites', 'court', 'codex_articles', 'defendants_simple', 'penalty', 'penalty_type', 'penalty_value', 'result_type', 'result_text', 'result_text_url', 'court_city', 'region', 'type', 'stage', 'url', 'appeal_date', 'defendants_gender', 'judge', 'case_number', 'last_scheduled_hearing_date', 'sm_url', 'latitude', 'longitude']


In [ ]:
import pandas as pd


df = pd.read_csv('/content/final_clean.csv')

columns_to_keep = [
    'id', 'predicted_result_type', 'sentence_year', 'sentence_months',
    'sentence_days', 'sentence_hours', 'is_suspended', 'suspension_years',
    'suspension_months', 'fine_is_fixed', 'fine_amount', 'drug_weight',
    'entry_date', 'result_date', 'court', 'codex_articles',
    'result_type', 'region', 'defendants_gender'
]

df_trimmed = df[columns_to_keep]


df_trimmed.to_csv('/content/clean_noresult.csv', index=False)



In [ ]:
print(df_trimmed['defendants_gender'].value_counts(dropna=False))

defendants_gender
М                                     8254
Ж                                      770
-                                      442
М, М                                   179
NaN                                     66
М, Ж                                    22
Ж, М                                    16
М, М, М                                 14
М, -                                    11
-, М                                     9
Ж, Ж                                     7
М, М, М, М, М                            4
-, -                                     3
М, М, М, М                               3
Ж, М, М                                  2
М, -, М                                  2
М, Ж, М                                  2
М, М, Ж, М                               2
М, -, Ж                                  1
М, М, М, М, М, М                         1
М, М, Ж                                  1
М, -, М, Ж, М, Ж, М, М, -, М, Ж, Ж       1
М, М, М, М, М, М, М                 

In [ ]:
import pandas as pd

df = pd.read_csv('/content/clean_noresult.csv')


def normalize_gender(value):
    if pd.isna(value) or value.strip() == '':
        return 'неизвестно'

    parts = [x.strip().lower() for x in value.split(',')]

    genders = set()
    for p in parts:
        if p in ['м', 'm', 'муж']:
            genders.add('муж')
        elif p in ['ж', 'ж.', 'w', 'жен']:
            genders.add('жен')
        elif p in ['-', '–', '—']:
            continue
        else:
            genders.add('неизвестно')

    if not genders:
        return 'неизвестно'
    elif genders == {'муж'}:
        return 'муж'
    elif genders == {'жен'}:
        return 'жен'
    elif 'неизвестно' in genders and len(genders) == 1:
        return 'неизвестно'
    else:
        return 'смешанный'


df['gender_group'] = df['defendants_gender'].apply(normalize_gender)
df.drop(columns=['defendants_gender'], inplace=True)
df.to_csv('/content/clean_noresu222lt.csv', index=False)


✅ Файл сохранён: clean_noresult_with_gender.csv


In [ ]:
import pandas as pd

df = pd.read_csv('/content/clean_noresu222lt.csv')


In [ ]:
print(" Типы данных:")
print(df.dtypes)

print("\n Пропуски по столбцам:")
print(df.isnull().sum())

print("\n Размер датафрейма:")
print(f"{df.shape[0]} строк, {df.shape[1]} колонок")


👀 Типы данных:
id                        object
predicted_result_type     object
sentence_year            float64
sentence_months          float64
sentence_days            float64
sentence_hours           float64
is_suspended             float64
suspension_years         float64
suspension_months        float64
fine_is_fixed            float64
fine_amount              float64
drug_weight               object
entry_date                object
result_date               object
court                     object
codex_articles            object
result_type               object
region                    object
gender_group              object
dtype: object

🧼 Пропуски по столбцам:
id                          0
predicted_result_type      35
sentence_year              35
sentence_months            35
sentence_days              35
sentence_hours             35
is_suspended               35
suspension_years           35
suspension_months          35
fine_is_fixed              35
fine_amount        

In [ ]:
df['drug_weight'] = (
    df['drug_weight']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .str.extract(r'(\d+(?:\.\d+)?)')[0]
    .astype(float)
)


In [ ]:
df['is_suspended'] = df['is_suspended'].astype('boolean')
df['fine_is_fixed'] = df['fine_is_fixed'].astype('boolean')


In [ ]:
df['gender_group'] = df['gender_group'].astype('category')


In [ ]:
print("\n Описание числовых колонок:")
print(df.describe())



📊 Описание числовых колонок:
       sentence_year  sentence_months  sentence_days  sentence_hours  \
count    9783.000000      9783.000000    9783.000000     9783.000000   
mean        0.856690        20.765818       6.941327       17.377389   
std         1.441104       645.377884     394.899410       68.020506   
min         0.000000         0.000000       0.000000        0.000000   
25%         0.000000         0.000000       0.000000        0.000000   
50%         0.000000         0.000000       0.000000        0.000000   
75%         1.000000         0.000000       0.000000        0.000000   
max        40.000000     30000.000000   30000.000000      480.000000   

       suspension_years  suspension_months    fine_amount  drug_weight  
count       9783.000000        9783.000000    9783.000000  6511.000000  
mean           0.502504           0.254114    2569.389758    82.238983  
std            1.103223           1.160512    9835.995227   387.060635  
min            0.000000      

In [ ]:
print("\nКатегориальные поля:")
for col in df.columns:
    if df[col].nunique() <= 20:
        print(f"\n🧩 {col} ({df[col].nunique()} уникальных):")
        print(df[col].value_counts(dropna=False))



🔠 Категориальные поля:

🧩 predicted_result_type (6 уникальных):
predicted_result_type
штраф                    4445
условное осуждение       3530
лишение свободы          1203
обязательные работы       599
NaN                        35
исправительные работы       4
ограничение свободы         2
Name: count, dtype: int64

🧩 sentence_year (14 уникальных):
sentence_year
0.0     6253
3.0     1755
1.0     1332
4.0      181
2.0      161
NaN       35
6.0       34
5.0       21
8.0       18
10.0      10
7.0       10
9.0        4
14.0       2
40.0       1
11.0       1
Name: count, dtype: int64

🧩 is_suspended (2 уникальных):
is_suspended
False    6817
True     2966
<NA>       35
Name: count, dtype: Int64

🧩 suspension_years (7 уникальных):
suspension_years
0.0     7396
2.0      832
1.0      794
3.0      641
4.0      110
NaN       35
5.0        9
50.0       1
Name: count, dtype: int64

🧩 suspension_months (11 уникальных):
suspension_months
0.0     9154
6.0      277
1.0      190
2.0       64
3.0 

In [ ]:
import pandas as pd


df = pd.read_csv('/content/clean_noresu222lt.csv')

df['drug_weight'] = (
    df['drug_weight']
    .astype(str)
    .str.replace(',', '.', regex=False)
    .str.extract(r'(\d+(?:\.\d+)?)')[0]
    .astype(float)
)

df['is_suspended'] = df['is_suspended'].astype('boolean')
df['fine_is_fixed'] = df['fine_is_fixed'].astype('boolean')

df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')
df['result_date'] = pd.to_datetime(df['result_date'], errors='coerce')

df['gender_group'] = df['gender_group'].astype('category')

df.to_csv('/content/clean_noresult_final.csv', index=False)

<ipython-input-46-e7fcb76c300a>:21: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['entry_date'] = pd.to_datetime(df['entry_date'], errors='coerce')


✅ Готово! Финальный датасет сохранён: clean_noresult_final.csv
